### 如何从原模型的tokenizer添加新的token

In [ ]:
#安装依赖，根据你的环境选择一种安装方法
# !pip install datasets evaluate 'transformers[sentencepiece]' #如果你的shell是zsh
# !pip install datasets evaluate 'transformers[sentencepiece]' #如果你的shell是bash

In [10]:
from datasets import load_from_disk

In [83]:
# 读取第一步生成datasets
file_path = "../data/hdnj"
raw_datasets = load_from_disk(file_path)

In [84]:
def get_training_corpus():
    return (
        raw_datasets [i: i + 1000]["text"]
        for i in range(0, len(raw_datasets), 1000)
    )


training_corpus = get_training_corpus()

# meta官方的llama需要申请后才能使用
首先需要注册一个hugging face的账号
去下面这个地址申请访问权限 https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

In [16]:
# 登入hugging face
from huggingface_hub import notebook_login

notebook_login()

In [74]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("ziqingyang/chinese-alpaca-2-7b")

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


# 我们来看看llama原模型的vacab_size

In [82]:
old_tokenizer

LlamaTokenizerFast(name_or_path='ziqingyang/chinese-alpaca-2-7b', vocab_size=55296, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

# vocab_size=32000 

In [76]:
example = '''
第一章上古天真论篇
昔在黄帝，生而神灵，弱而能言，幼而徇齐，长而敦敏，成而登天。
乃问于天师曰：余闻上古之人，春秋皆度百岁，而动作不衰；今时之人，年半百而动作皆衰者。时世异耶人将失之耶？
岐伯对曰：上古之人？其知道者，法于阴阳，和于术数，食饮有节，起居有常，不妄作劳，故能形与神俱，而尽终其天年，度百岁乃去。
今时之人不然也，以酒为浆，以妄为常，醉以入房，以欲竭其精，以耗散其真，不知持满，不时御神，务快其心，逆于生乐，起居无节，故半百而衰也。
夫上古圣人之教下也，皆谓之虚邪贼风避之有时，恬惔虚无，真气从之，精神内守，病安从来。
是以志闲而少欲，心安而不惧，形劳而不倦，气从以顺，各从其欲，皆得所愿。故美其食，任其服，乐其俗，高下不相慕，其民故曰朴。
是以嗜欲不能劳其目，淫邪不能惑其心，愚智贤不肖，不惧于物，故合于道。
所以能年皆度百岁而动作不衰者，以其德全不危也。
帝曰：人年老而无子者，材力尽邪？将天数然也？
岐伯曰：女子七岁肾气盛，齿更发长。
二七而天癸至，任脉通，太冲脉盛，月事以时下，故有子。
三七肾气平均，故真牙生而长极。
四七筋骨坚，发长极，身体盛壮。
五七阳明脉衰，面始焦，发始堕。
六七三阳脉衰于上，面皆焦，发始白。
七七任脉虚，太冲脉衰少，天癸竭，地道不通，故形坏而无子也。
丈夫八岁肾气实，发长齿更。
二八肾气盛，天癸至，精气溢泻，阴阳和，故能有子。
三八肾气平均，筋骨劲强，故真牙生而长极。
四八筋骨隆盛，肌肉满壮。
五八肾气衰，发堕齿槁。
六八阳气衰竭于上，面焦，发鬓颁白。
七八肝气衰，筋不能动，天癸竭，精少，肾脏衰，形体皆极。
八八则齿发去。
肾者主水，受五脏六腑之精而藏之，故五脏盛，乃能泻。
今五脏皆衰，筋骨解堕，天癸尽矣，故发鬓白，身体重，行步不正，而无子耳。
帝曰：有其年已老，而有子者：何也？岐伯曰：此其天寿过度，气脉常通，而肾气有余也。此虽有子，男子不过尽八八，女子不过尽七七，而天地之精气皆竭矣。
帝曰：夫道者年皆百岁，能有子乎？岐伯曰：夫道者能却老而全角，身年虽寿，能生子也。
黄帝曰：余闻上古有真人者，提挈天地，把握阴阳，呼吸精气，独立守神，肌肉若一，故能寿敝天地，无有终时，此其道生。
中古之时，有至人者，淳德全道，和于阴阳，调于四时，去世离俗，积精全神，游行天地之间，视听八远之外，此盖益其寿命而强者也。亦归于真人。
其次有圣人者，处天地之和，从八风之理，适嗜欲于世俗之间，无恚嗔之心，行不欲离于世，被服章，举不欲观于俗，外不劳形于事，内无思想之患，以恬愉为务，以自得为功，形体不敝，精神不散，亦可以百数。
其次有贤人者，法则天地，象似日月，辨列星辰，逆从阴阳，分别四时，将从上古合同于道，亦可使益寿而有极时。'''

In [77]:
tokens = old_tokenizer.tokenize(example)
tokens

['▁',
 '<0x0A>',
 '第一',
 '章',
 '上',
 '古',
 '天真',
 '论',
 '篇',
 '<0x0A>',
 '昔',
 '在',
 '黄',
 '帝',
 '，',
 '生',
 '而',
 '神',
 '灵',
 '，',
 '弱',
 '而',
 '能',
 '言',
 '，',
 '幼',
 '而',
 '徇',
 '齐',
 '，',
 '长',
 '而',
 '敦',
 '敏',
 '，',
 '成',
 '而',
 '登',
 '天',
 '。',
 '<0x0A>',
 '乃',
 '问',
 '于',
 '天',
 '师',
 '曰',
 '：',
 '余',
 '闻',
 '上',
 '古',
 '之人',
 '，',
 '春秋',
 '皆',
 '度',
 '百',
 '岁',
 '，',
 '而',
 '动作',
 '不',
 '衰',
 '；',
 '今',
 '时',
 '之人',
 '，',
 '年',
 '半',
 '百',
 '而',
 '动作',
 '皆',
 '衰',
 '者',
 '。',
 '时',
 '世',
 '异',
 '耶',
 '人',
 '将',
 '失',
 '之',
 '耶',
 '？',
 '<0x0A>',
 '岐',
 '伯',
 '对',
 '曰',
 '：',
 '上',
 '古',
 '之人',
 '？',
 '其',
 '知道',
 '者',
 '，',
 '法',
 '于',
 '阴阳',
 '，',
 '和',
 '于',
 '术',
 '数',
 '，',
 '食',
 '饮',
 '有',
 '节',
 '，',
 '起',
 '居',
 '有',
 '常',
 '，',
 '不',
 '妄',
 '作',
 '劳',
 '，',
 '故',
 '能',
 '形',
 '与',
 '神',
 '俱',
 '，',
 '而',
 '尽',
 '终',
 '其',
 '天',
 '年',
 '，',
 '度',
 '百',
 '岁',
 '乃',
 '去',
 '。',
 '<0x0A>',
 '今',
 '时',
 '之人',
 '不然',
 '也',
 '，',
 '以',
 '酒',
 '为',
 '浆',
 '，',
 '以',
 '妄',
 '为

## 可以看出中文的token是按照utf-8字符串作为编码的

In [ ]:
# 通过原模型的token来训练datasets里面数据的token
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 72000)

In [88]:
tokenizer.is_fast

True

In [86]:
tokenizer

LlamaTokenizerFast(name_or_path='ziqingyang/chinese-alpaca-2-7b', vocab_size=53698, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [87]:
tokens = tokenizer.tokenize(example)
tokens

['▁',
 '<unk>',
 '第',
 '一章',
 '上古天真',
 '论篇',
 '<unk>',
 '昔',
 '在黄帝',
 '，生而神灵，弱而能言，',
 '幼而徇齐，长而敦敏，成而登天。',
 '<unk>',
 '乃',
 '问于天',
 '师曰：余闻上古之人，',
 '春秋皆度百岁，而动作不衰；今时之人，年半百而动作皆衰者。时世异耶人将失之耶？',
 '<unk>',
 '岐伯对曰：',
 '上古',
 '之人？其知道',
 '者，法于阴阳，和于术数，食饮有节，起居有常，不妄作劳',
 '，故能形与神俱，而尽终其天年，度百岁乃去。',
 '<unk>',
 '今时之',
 '人不然也，以酒为浆',
 '，以妄为常，醉以入房，以欲竭其精，以耗散其真，不知持满，不时御神，务快其心，逆于',
 '生乐，起居无节，故半百而衰也。',
 '<unk>',
 '夫上古',
 '圣人之教下也，皆',
 '谓之虚邪贼风避之有时，恬惔虚无，真气',
 '从之，精神内守，病安从来。',
 '<unk>',
 '是以',
 '志闲',
 '而少欲，心',
 '安而不惧，形劳而不倦，气从以顺，各从其欲，皆得所愿。故',
 '美其食，任其服，乐其俗，高下不相慕，其民故曰朴。',
 '<unk>',
 '是以',
 '嗜欲',
 '不能劳其目，淫邪不能惑其心，',
 '愚智贤不肖，不惧于物，故合于',
 '道。',
 '<unk>',
 '所以',
 '能年皆',
 '度百岁而动作不衰者，以其德',
 '全不危也。',
 '<unk>',
 '帝曰：',
 '人',
 '年老',
 '而无子者，',
 '材力尽邪？将天数然也？',
 '<unk>',
 '岐伯曰：',
 '女子',
 '七岁肾气盛，齿更',
 '发长。',
 '<unk>',
 '二',
 '七而',
 '天癸至，任脉通，太',
 '冲脉盛，月事以时下，故有子。',
 '<unk>',
 '三',
 '七肾气',
 '平均，故真牙生而长极。',
 '<unk>',
 '四',
 '七筋骨',
 '坚，发长极，身体',
 '盛壮。',
 '<unk>',
 '五',
 '七阳明',
 '脉衰，面始焦，发始',
 '堕。',
 '<unk>',
 '六',
 '七三阳',
 '脉衰于上，面皆焦，发始',
 '

In [45]:
len(tokens)

1